In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

from result_analysis import read_pd

In [2]:
PATH_TO_DATA = Path('../../data/experiments/aggregate_results').resolve()
expert_f = PATH_TO_DATA.joinpath('12-04-2021/14_04_08')
summary_f = PATH_TO_DATA.joinpath('12-05-2021/10_26_23')
gpt_f = PATH_TO_DATA.joinpath('12-05-2021/13_46_24')
studio_f = PATH_TO_DATA.joinpath('02-02-2022/10_07_59')
PATH_TO_DATA

PosixPath('/media/HD/Documents/Prompt-Summarization/data/experiments/aggregate_results')

In [3]:
def get_orig_better_worse_same(path):
    fname = path.joinpath('better_df.csv')
    better = read_pd(fname)

    fname = path.joinpath('same_df.csv')
    same = read_pd(fname)

    fname = path.joinpath('worse_df.csv')
    worse = read_pd(fname)

    fname = path.joinpath('original_df.csv')
    orig = read_pd(
        fname,
        usecols=[0, 1, 2, 3, 4],
        header=0,
        names=["Problem Names", -2, -1, False, True]
    )
    return orig, better, worse, same

In [4]:
summary_orig, summary_better, summary_worse, summary_same = get_orig_better_worse_same(summary_f)
expert_orig, expert_better, expert_worse, expert_same = get_orig_better_worse_same(expert_f)
gpt_orig, gpt_better, gpt_worse, gpt_same = get_orig_better_worse_same(gpt_f)
studio_orig, studio_better, studio_worse, studio_same = get_orig_better_worse_same(studio_f)

In [5]:
print(summary_orig.shape, summary_better.shape, summary_worse.shape, summary_same.shape)
print(expert_orig.shape, expert_better.shape, expert_worse.shape, expert_same.shape)
print(gpt_orig.shape, gpt_better.shape, gpt_worse.shape, gpt_same.shape)
print(studio_orig.shape, studio_better.shape, studio_worse.shape, studio_same.shape)

(859, 4) (75, 6) (85, 6) (355, 4)
(859, 4) (101, 6) (87, 6) (334, 4)
(1000, 4) (177, 6) (174, 6) (355, 4)
(1991, 4) (264, 6) (338, 6) (745, 4)


In [6]:
gpt_orig

,-2,-1,False,True
Problem Names,,,,
gpt_generated/test/competition/3845/expert.txt,0,44,0,0
gpt_generated/test/competition/3845/question.txt,1,0,0,0
gpt_generated/test/competition/3799/expert.txt,0,0,20,27
gpt_generated/test/competition/3799/question.txt,0,0,22,25
gpt_generated/test/interview/2927/expert.txt,0,2,0,0
...,...,...,...,...
gpt_generated/test/interview/2828/question.txt,0,0,3,0
gpt_generated/test/interview/2794/expert.txt,0,0,3,0
gpt_generated/test/interview/2794/question.txt,0,0,3,0


In [7]:
expert_orig

,-2,-1,False,True
Problem Names,,,,
human_generated/train/introductory/4025/expert.txt,0,0,5,0
human_generated/train/introductory/4025/question.txt,0,0,5,0
human_generated/train/interview/0990/expert.txt,0,1,0,0
human_generated/train/interview/0990/question.txt,0,1,0,0
human_generated/train/competition/2296/expert.txt,0,0,2,1
...,...,...,...,...
human_generated/train/interview/0030/question.txt,0,0,1,0
human_generated/train/competition/2005/expert.txt,0,0,9,10
human_generated/train/competition/2005/question.txt,0,0,9,10


In [8]:
def remove_generated(x):
    arr = x.split('/')[1:]
    return '/'.join(arr)

def remove_file(x):
    arr = x.split('/')[:-1]
    return '/'.join(arr)

def add_generated(x, type):
    return f'{type}_generated/{x}'

def get_problem(x):
    x = remove_generated(x)
    return remove_file(x)

human_idx = expert_orig.index.map(remove_generated)

In [9]:
gpt_human_idx = human_idx.map(lambda x: add_generated(x, 'gpt'))
human_and_gpt_idx = gpt_human_idx.intersection(gpt_orig.index)
print(gpt_human_idx.shape, human_and_gpt_idx.shape)
temp = gpt_orig.copy(deep=False)
temp.loc[human_and_gpt_idx]

(859,) (0,)


,-2,-1,False,True
Problem Names,,,,


# Get the problems that human's summarized and Studio21 generated

In [10]:
studio_human_idx = human_idx.map(lambda x: add_generated(x, 'studio21'))
human_and_studio_idx = studio_human_idx.intersection(studio_orig.index)
temp_studio = studio_orig.copy(deep=False)
temp_studio = temp_studio.loc[human_and_studio_idx]

In [11]:
collision_idx = temp_studio.index.map(remove_generated)
collision_idx = collision_idx.map(lambda x: add_generated(x, 'human'))

In [12]:
temp_expert = expert_orig.loc[collision_idx]

In [13]:
def score_prob(x):
    if x[-2] > 0:
        return -np.inf
    return -x[-1] + x[1]

# Remove the generated prefix so we can merge on problem names

In [14]:
temp_expert['score'] = temp_expert.apply(score_prob, axis=1)
temp_studio['score'] = temp_studio.apply(score_prob, axis=1)
temp_expert['Prob'] = temp_expert.index.map(remove_generated)
temp_studio['Prob'] = temp_studio.index.map(remove_generated)

In [15]:
temp_expert.head()

,-2,-1,False,True,score,Prob
Problem Names,,,,,,
human_generated/train/interview/0990/expert.txt,0,1,0,0,-1.0,train/interview/0990/expert.txt
human_generated/train/interview/0990/question.txt,0,1,0,0,-1.0,train/interview/0990/question.txt
human_generated/train/competition/2296/expert.txt,0,0,2,1,1.0,train/competition/2296/expert.txt
human_generated/train/competition/2296/question.txt,0,0,2,1,1.0,train/competition/2296/question.txt
human_generated/train/interview/1502/expert.txt,0,0,0,1,1.0,train/interview/1502/expert.txt


In [16]:
studio_expert_merge = pd.merge(temp_studio, temp_expert, how='inner', on='Prob', suffixes=['_studio21', '_expert'])
studio_expert_merge 

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_expert,-1_expert,False_expert,True_expert,score_expert
0,0,1,0,0,-1.0,train/interview/0990/expert.txt,0,1,0,0,-1.0
1,0,1,0,0,-1.0,train/interview/0990/question.txt,0,1,0,0,-1.0
2,0,0,2,1,1.0,train/competition/2296/expert.txt,0,0,2,1,1.0
3,0,0,1,2,2.0,train/competition/2296/question.txt,0,0,2,1,1.0
4,0,0,0,1,1.0,train/interview/1502/expert.txt,0,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
617,0,0,1,0,0.0,train/interview/0030/question.txt,0,0,1,0,0.0
618,0,0,9,10,10.0,train/competition/2005/expert.txt,0,0,9,10,10.0
619,0,0,9,10,10.0,train/competition/2005/question.txt,0,0,9,10,10.0
620,0,0,4,0,0.0,train/competition/2248/expert.txt,0,1,3,0,-1.0


In [17]:
def check_same_results(x):
    studio = x[:4]
    expert = x[6:-1]
    return (studio.values==expert.values).all()

In [29]:
expert_diff_results = studio_expert_merge.loc[~studio_expert_merge.apply(check_same_results, axis=1)]
expert_diff_results 

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_expert,-1_expert,False_expert,True_expert,score_expert
3,0,0,1,2,2.0,train/competition/2296/question.txt,0,0,2,1,1.0
6,1,0,0,0,-inf,train/introductory/4159/expert.txt,0,0,0,3,3.0
7,1,0,0,0,-inf,train/introductory/4159/question.txt,0,0,2,1,1.0
8,0,0,12,5,5.0,train/competition/2032/expert.txt,0,17,0,0,-17.0
9,0,0,14,3,3.0,train/competition/2032/question.txt,0,17,0,0,-17.0
...,...,...,...,...,...,...,...,...,...,...,...
608,0,0,1,0,0.0,train/interview/1380/expert.txt,0,0,0,1,1.0
612,0,0,11,3,3.0,train/competition/2071/expert.txt,0,0,12,2,2.0
614,0,4,0,5,1.0,train/introductory/3028/expert.txt,0,0,0,9,9.0
615,0,0,8,1,1.0,train/introductory/3028/question.txt,0,3,0,6,3.0


In [32]:
studio_better_than_expert = expert_diff_results[expert_diff_results['score_studio21'] > expert_diff_results['score_expert']]
studio_better_than_expert 

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_expert,-1_expert,False_expert,True_expert,score_expert
3,0,0,1,2,2.0,train/competition/2296/question.txt,0,0,2,1,1.0
8,0,0,12,5,5.0,train/competition/2032/expert.txt,0,17,0,0,-17.0
9,0,0,14,3,3.0,train/competition/2032/question.txt,0,17,0,0,-17.0
10,0,0,14,1,1.0,train/competition/2057/expert.txt,0,8,4,3,-5.0
13,0,0,14,0,0.0,train/competition/2063/question.txt,1,0,0,0,-inf
...,...,...,...,...,...,...,...,...,...,...,...
576,0,1,0,0,-1.0,train/interview/0012/expert.txt,1,0,0,0,-inf
586,0,0,17,2,2.0,train/competition/2004/expert.txt,0,0,18,1,1.0
596,0,0,0,1,1.0,train/interview/0493/expert.txt,0,1,0,0,-1.0
612,0,0,11,3,3.0,train/competition/2071/expert.txt,0,0,12,2,2.0


In [33]:
expert_better_than_studio = expert_diff_results[expert_diff_results['score_studio21'] < expert_diff_results['score_expert']]
expert_better_than_studio 

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_expert,-1_expert,False_expert,True_expert,score_expert
6,1,0,0,0,-inf,train/introductory/4159/expert.txt,0,0,0,3,3.0
7,1,0,0,0,-inf,train/introductory/4159/question.txt,0,0,2,1,1.0
12,0,0,14,0,0.0,train/competition/2063/expert.txt,0,0,12,2,2.0
16,0,0,1,0,0.0,train/interview/1540/expert.txt,0,0,0,1,1.0
29,0,0,1,0,0.0,train/interview/1087/question.txt,0,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
604,0,0,4,0,0.0,train/introductory/4059/expert.txt,0,0,0,4,4.0
606,0,0,1,0,0.0,train/interview/0036/expert.txt,0,0,0,1,1.0
608,0,0,1,0,0.0,train/interview/1380/expert.txt,0,0,0,1,1.0
614,0,4,0,5,1.0,train/introductory/3028/expert.txt,0,0,0,9,9.0


# Now do the same for Summary.txt

In [21]:
def change_summary_to_expert(x):
    return x.replace('summary', 'expert')

def change_expert_to_summary(x):
    return x.replace('expert', 'summary')


In [22]:
collision_idx_summary = collision_idx.map(change_expert_to_summary)
temp_summary = summary_orig.loc[collision_idx_summary]
temp_summary.shape

(622, 4)

In [23]:
temp_summary['score'] = temp_summary.apply(score_prob, axis=1)
temp_summary['Prob'] = temp_summary.index.map(remove_generated).map(change_summary_to_expert)

In [24]:
temp_summary.head()

,-2,-1,False,True,score,Prob
Problem Names,,,,,,
human_generated/train/interview/0990/summary.txt,0,1,0,0,-1.0,train/interview/0990/expert.txt
human_generated/train/interview/0990/question.txt,0,1,0,0,-1.0,train/interview/0990/question.txt
human_generated/train/competition/2296/summary.txt,0,0,2,1,1.0,train/competition/2296/expert.txt
human_generated/train/competition/2296/question.txt,0,0,2,1,1.0,train/competition/2296/question.txt
human_generated/train/interview/1502/summary.txt,0,0,0,1,1.0,train/interview/1502/expert.txt


In [25]:
studio_summary_merge = pd.merge(temp_studio, temp_summary, how='inner', on='Prob', suffixes=['_studio21', '_summary'])
studio_summary_merge 

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_summary,-1_summary,False_summary,True_summary,score_summary
0,0,1,0,0,-1.0,train/interview/0990/expert.txt,0,1,0,0,-1.0
1,0,1,0,0,-1.0,train/interview/0990/question.txt,0,1,0,0,-1.0
2,0,0,2,1,1.0,train/competition/2296/expert.txt,0,0,2,1,1.0
3,0,0,1,2,2.0,train/competition/2296/question.txt,0,0,2,1,1.0
4,0,0,0,1,1.0,train/interview/1502/expert.txt,0,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
617,0,0,1,0,0.0,train/interview/0030/question.txt,0,0,1,0,0.0
618,0,0,9,10,10.0,train/competition/2005/expert.txt,0,0,9,10,10.0
619,0,0,9,10,10.0,train/competition/2005/question.txt,0,0,9,10,10.0
620,0,0,4,0,0.0,train/competition/2248/expert.txt,0,1,3,0,-1.0


In [34]:
summary_diff_results = studio_summary_merge.loc[~studio_summary_merge.apply(check_same_results, axis=1)]
summary_diff_results 

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_summary,-1_summary,False_summary,True_summary,score_summary
3,0,0,1,2,2.0,train/competition/2296/question.txt,0,0,2,1,1.0
6,1,0,0,0,-inf,train/introductory/4159/expert.txt,0,0,0,3,3.0
7,1,0,0,0,-inf,train/introductory/4159/question.txt,0,0,1,2,2.0
8,0,0,12,5,5.0,train/competition/2032/expert.txt,0,17,0,0,-17.0
9,0,0,14,3,3.0,train/competition/2032/question.txt,0,17,0,0,-17.0
...,...,...,...,...,...,...,...,...,...,...,...
608,0,0,1,0,0.0,train/interview/1380/expert.txt,1,0,0,0,-inf
612,0,0,11,3,3.0,train/competition/2071/expert.txt,0,0,0,14,14.0
614,0,4,0,5,1.0,train/introductory/3028/expert.txt,0,3,0,6,3.0
615,0,0,8,1,1.0,train/introductory/3028/question.txt,0,3,0,6,3.0


In [35]:
studio_better_than_summary = summary_diff_results[summary_diff_results['score_studio21'] > summary_diff_results['score_summary']]
studio_better_than_summary 

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_summary,-1_summary,False_summary,True_summary,score_summary
3,0,0,1,2,2.0,train/competition/2296/question.txt,0,0,2,1,1.0
8,0,0,12,5,5.0,train/competition/2032/expert.txt,0,17,0,0,-17.0
9,0,0,14,3,3.0,train/competition/2032/question.txt,0,17,0,0,-17.0
13,0,0,14,0,0.0,train/competition/2063/question.txt,0,9,5,0,-9.0
19,0,0,6,1,1.0,train/competition/2198/question.txt,0,0,7,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
593,0,0,0,3,3.0,train/introductory/4102/question.txt,0,3,0,0,-3.0
596,0,0,0,1,1.0,train/interview/0493/expert.txt,1,0,0,0,-inf
601,0,13,3,2,-11.0,train/competition/2010/question.txt,0,15,3,0,-15.0
608,0,0,1,0,0.0,train/interview/1380/expert.txt,1,0,0,0,-inf


In [36]:
summary_better_than_studio = summary_diff_results[summary_diff_results['score_studio21'] < summary_diff_results['score_summary']]
summary_better_than_studio 

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_summary,-1_summary,False_summary,True_summary,score_summary
6,1,0,0,0,-inf,train/introductory/4159/expert.txt,0,0,0,3,3.0
7,1,0,0,0,-inf,train/introductory/4159/question.txt,0,0,1,2,2.0
12,0,0,14,0,0.0,train/competition/2063/expert.txt,0,0,12,2,2.0
29,0,0,1,0,0.0,train/interview/1087/question.txt,0,0,0,1,1.0
34,0,1,0,0,-1.0,train/interview/1403/expert.txt,0,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
604,0,0,4,0,0.0,train/introductory/4059/expert.txt,0,0,0,4,4.0
606,0,0,1,0,0.0,train/interview/0036/expert.txt,0,0,0,1,1.0
612,0,0,11,3,3.0,train/competition/2071/expert.txt,0,0,0,14,14.0
614,0,4,0,5,1.0,train/introductory/3028/expert.txt,0,3,0,6,3.0


In [37]:
studio_better_than_expert.head()

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_expert,-1_expert,False_expert,True_expert,score_expert
3,0,0,1,2,2.0,train/competition/2296/question.txt,0,0,2,1,1.0
8,0,0,12,5,5.0,train/competition/2032/expert.txt,0,17,0,0,-17.0
9,0,0,14,3,3.0,train/competition/2032/question.txt,0,17,0,0,-17.0
10,0,0,14,1,1.0,train/competition/2057/expert.txt,0,8,4,3,-5.0
13,0,0,14,0,0.0,train/competition/2063/question.txt,1,0,0,0,-inf


In [38]:
studio_better_than_summary.head()

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_summary,-1_summary,False_summary,True_summary,score_summary
3,0,0,1,2,2.0,train/competition/2296/question.txt,0,0,2,1,1.0
8,0,0,12,5,5.0,train/competition/2032/expert.txt,0,17,0,0,-17.0
9,0,0,14,3,3.0,train/competition/2032/question.txt,0,17,0,0,-17.0
13,0,0,14,0,0.0,train/competition/2063/question.txt,0,9,5,0,-9.0
19,0,0,6,1,1.0,train/competition/2198/question.txt,0,0,7,0,0.0


In [42]:
studio_better_than_human = set(studio_better_than_summary['Prob']) & set(studio_better_than_expert['Prob'])

In [50]:
studio_better_than_human = pd.Series(list(studio_better_than_human))
studio_probs_better_than_human = studio_better_than_human.apply(remove_file)
studio_probs_better_than_human.to_csv('studio_probs_better_than_human.csv', index=False, header=False)

In [54]:
expert_same.loc['human_generated/train/competition/2004']

-2=>-2           0
-1=>-1           0
False=>False    10
True=>True       1
Name: human_generated/train/competition/2004, dtype: int64

In [55]:
expert_worse.loc['human_generated/train/competition/2004']

-1=>-2         0
False=>-2      0
False=>-1      0
True=>-2       0
True=>-1       0
True=>False    8
Name: human_generated/train/competition/2004, dtype: int64

In [56]:
from result_analysis import main

In [57]:
expert_results = main('12-04-2021/14_04_08')
summary_results = main('12-05-2021/10_26_23')
gpt_results = main('12-05-2021/13_46_24')
studio_results = main('02-02-2022/10_07_59')

Total Accuracy: 36.32%
Strict Accuracy: 31.14%
Total Accuracy: 35.39%
Strict Accuracy: 30.82%
Total Accuracy: 14.32%
Strict Accuracy: 2.88%
Total Accuracy: 24.77%
Strict Accuracy: 17.82%


In [79]:
studio_probs_better_than_human = pd.read_csv('sorted_studio_better.csv', header=None)
studio_probs_human_idx = studio_probs_better_than_human[0].apply(lambda x: add_generated(x, 'human')).unique()
studio_probs_studio_idx = studio_probs_better_than_human[0].apply(lambda x: add_generated(x, 'studio21')).unique()

In [81]:
expert_results.loc[studio_probs_human_idx[:5]]

,Original Len,Summary Len,Summarization Percent,Code Solution Min Len,Code Solution Mean Len,Original Code Generated Len,Summary Code Generated Len,Test Cases,Original Accuracy,Summary Accuracy,Accuracy Increase
Problem Names,,,,,,,,,,,
human_generated/train/competition/2004,1276,485,61.990596,52,187.760000,271,210,11.0,47.368421,5.263158,-42.105263
human_generated/train/competition/2012,795,650,18.238994,188,468.583333,508,470,18.0,11.111111,11.111111,0.000000
human_generated/train/competition/2017,1495,857,42.675585,112,275.750000,232,169,18.0,5.555556,5.555556,0.000000
human_generated/train/competition/2032,1605,1162,27.601246,1492,1781.250000,568,442,17.0,0.000000,0.000000,0.000000
human_generated/train/competition/2033,2158,1057,51.019462,282,346.000000,2221,4464,2.0,11.764706,11.764706,0.000000


In [86]:
summary_results.loc[studio_probs_human_idx[:5]]

,Original Len,Summary Len,Summarization Percent,Code Solution Min Len,Code Solution Mean Len,Original Code Generated Len,Summary Code Generated Len,Test Cases,Original Accuracy,Summary Accuracy,Accuracy Increase
Problem Names,,,,,,,,,,,
human_generated/train/competition/2004,1276,554,56.583072,52,187.760000,219,179,18.0,10.526316,5.263158,-5.263158
human_generated/train/competition/2012,795,795,0.000000,188,468.583333,821,932,18.0,11.111111,16.666667,5.555556
human_generated/train/competition/2017,1495,955,36.120401,112,275.750000,231,169,18.0,5.555556,5.555556,0.000000
human_generated/train/competition/2032,1605,1227,23.551402,1492,1781.250000,380,518,17.0,0.000000,0.000000,0.000000
human_generated/train/competition/2033,2158,1382,35.959222,282,346.000000,2262,918,11.0,11.764706,11.764706,0.000000


In [93]:
studio_results.loc[studio_probs_studio_idx[:5]]

,Original Len,Summary Len,Summarization Percent,Code Solution Min Len,Code Solution Mean Len,Original Code Generated Len,Summary Code Generated Len,Test Cases,Original Accuracy,Summary Accuracy,Accuracy Increase
Problem Names,,,,,,,,,,,
studio21_generated/train/competition/2004,1276,808,36.677116,52,187.760000,166,55,12.0,36.842105,10.526316,-26.315789
studio21_generated/train/competition/2012,818,534,34.718826,188,468.583333,526,501,18.0,55.555556,66.666667,11.111111
studio21_generated/train/competition/2017,1495,1447,3.210702,112,275.750000,176,235,18.0,5.555556,27.777778,22.222222
studio21_generated/train/competition/2032,1605,951,40.747664,1492,1781.250000,407,614,16.0,17.647059,29.411765,11.764706
studio21_generated/train/competition/2033,2158,1588,26.413346,282,346.000000,2347,2286,15.0,11.764706,0.000000,-11.764706


In [102]:
prob_to_investigate = 'train/competition/2012/expert.txt'
#prob_to_investigate = 'train/competition/2012/question.txt'

In [103]:
studio_better_than_expert[studio_better_than_expert['Prob'] == prob_to_investigate]

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_expert,-1_expert,False_expert,True_expert,score_expert
248,0,0,6,12,12.0,train/competition/2012/expert.txt,0,0,16,2,2.0


In [104]:
studio_better_than_summary[studio_better_than_summary['Prob'] == prob_to_investigate]

,-2_studio21,-1_studio21,False_studio21,True_studio21,score_studio21,Prob,-2_summary,-1_summary,False_summary,True_summary,score_summary
248,0,0,6,12,12.0,train/competition/2012/expert.txt,0,0,15,3,3.0
